In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#https://drive.google.com/open?id=1_Xo1PTfFgY3KgPVxo5gzghj_pRzp5gCj
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1_Xo1PTfFgY3KgPVxo5gzghj_pRzp5gCj'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("images_Data.zip")
# print('Downloaded content "{}"'.format(downloaded.GetContentString()))
!unzip images_Data.zip

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving augments_label.pickle to augments_label.pickle
Saving labels.pickle to labels.pickle
Saving model.py to model.py
Saving preprocessing.py to preprocessing.py
User uploaded file "augments_label.pickle" with length 123212 bytes
User uploaded file "labels.pickle" with length 33758 bytes
User uploaded file "model.py" with length 838 bytes
User uploaded file "preprocessing.py" with length 992 bytes


In [2]:
import os
print(len(os.listdir("./images")))
!cp -a ./aug_images/. ./images/
print(len(os.listdir("./images")))

1065


In [3]:
!ls

adc.json	       images		model.py	  sample_data
aug_images	       images_Data.zip	preprocessing.py
augments_label.pickle  labels.pickle	__pycache__


In [1]:
import keras as ks
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pickle as pkl

Using TensorFlow backend.


In [0]:
from model import create_model
from preprocessing import preprocess

In [3]:
orig_labels=[]
with open("./labels.pickle","rb") as file:
    orig_labels=pkl.load(file)
    
aug_labels=[]
with open("./augments_label.pickle","rb") as file:
    aug_labels=pkl.load(file)
    
labels=orig_labels+aug_labels
np.random.shuffle(labels)
len(labels)

1065

In [4]:
total_data=0
ones=[]
for image in labels:
    total_data+=image[1][0]
    ones+=image[1][-1]
total_data,(np.array(ones)=='1').sum()

(1920, 786)

In [0]:
import keras as ks
# import tensorflow as tf

def create_model(input_shape):
    base_model=ks.applications.VGG16(include_top=False,input_shape=input_shape)

    trainable=False
    for layer in base_model.layers:
        if layer.name == "block5_conv1":
            trainable=True
        layer.trainable=trainable

    model=ks.models.Sequential()
    model.add(base_model)
    model.add(ks.layers.Flatten())
    model.add(ks.layers.Dropout(rate=0.25))
    model.add(ks.layers.Dense(1024,activation="relu"))
    model.add(ks.layers.Dense(512,activation="relu"))
    model.add(ks.layers.Dense(128,activation="relu"))
    model.add(ks.layers.Dense(64,activation="relu"))
    model.add(ks.layers.Dense(1,activation="sigmoid"))

    model.compile(optimizer="adam",loss=ks.losses.binary_crossentropy,metrics=["accuracy"])

    return model

In [0]:
model=create_model((224,224,3))

In [7]:
batch_size=200
epochs=5
steps_per_epoch=int(total_data/(batch_size))
steps_per_epoch

9

In [0]:
model.fit_generator(preprocess(labels,"./images/",batch_size=batch_size),steps_per_epoch=steps_per_epoch,epochs=50)

Epoch 1/50
9/9 [==============================] - 60s 7s/step - loss: 1.0391 - acc: 0.5379
Epoch 2/50
9/9 [==============================] - 24s 3s/step - loss: 0.6914 - acc: 0.5859
Epoch 3/50
9/9 [==============================] - 29s 3s/step - loss: 0.6871 - acc: 0.6311
Epoch 4/50
9/9 [==============================] - 25s 3s/step - loss: 0.6868 - acc: 0.5795
Epoch 5/50
9/9 [==============================] - 18s 2s/step - loss: 0.6803 - acc: 0.6084
Epoch 6/50
1/9 [==>...........................] - ETA: 15s - loss: 0.6729 - acc: 0.6368

In [0]:
img=cv2.resize(cv2.imread("./aug_images/IMG-20190708-WA0008_4.jpg"),(224,224))
plt.imshow(img)

In [0]:
model.predict([[img]])

array([[1.]], dtype=float32)

In [0]:
del model